In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense,Activation,Dropout

Using TensorFlow backend.


In [2]:
import os
import numpy as np
from PIL import Image

### 1.字符识别

In [17]:
def img2arr_c(imgpath, _crop=None):
    img = Image.open(imgpath).convert("L")
    if _crop:
        img = img.crop(_crop)
    return np.array(img.getdata())/255

def onehot(st):
    u = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    arr = np.zeros(36, dtype="float")
    idx = u.find(st)
    arr[idx] = 1
    return arr

def hotone(_arr):
    u = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    idx = np.where(_arr==_arr.max())[0][0]
    return u[idx]

def load_data_c(path, n, size):
    datas = np.zeros([n, size])
    for i in range(n):
        arry = img2arr(path+"\\%d.png" %i)
        datas[i,:] = arry
    return datas

def load_labe_c(file, _onehot=True):
    with open(file) as f:
        text = f.read()
        lis_ = eval(text)
    n = len(lis_)
    labe = np.zeros([n, 36])
    if _onehot:
        for i,each in enumerate(lis_):
            labe[i, :] = onehot(each)
        return labe
    else:
        return lis_

In [ ]:
# model = Sequential()
# model.add(Dense(units=128, input_dim=864, activation="relu", use_bias=True))
# model.add(Dropout(0.5))
# model.add(Dense(units=128, activation="relu", use_bias=True))
# model.add(Dropout(0.5))
# model.add(Dense(units=36, activation="softmax"))
# model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [18]:
model_c = load_model("char.h5")

#### 训练数据
_data: 二值化，取反，kmeans切割后的字符图片(size=(24,36)),展平_

_labe: onehot型(size=36)_

In [19]:
data_c = load_data_c(r"..\Database\chars", n=2289, size=864)
labe_c = load_labe_c(r"..\Database\charslabel.txt")

In [20]:
model_c.fit(data_c, labe_c, epochs=100, batch_size=50)

Epoch 1/100
2289/2289 [==============================] - 0s - loss: 0.1343 - acc: 0.9616     
Epoch 2/100
2289/2289 [==============================] - 0s - loss: 0.1377 - acc: 0.9567     
Epoch 3/100
2289/2289 [==============================] - 0s - loss: 0.1419 - acc: 0.9559     
Epoch 4/100
2289/2289 [==============================] - 0s - loss: 0.1323 - acc: 0.9581     
Epoch 5/100
2289/2289 [==============================] - 0s - loss: 0.1148 - acc: 0.9655     
Epoch 6/100
2289/2289 [==============================] - 0s - loss: 0.1336 - acc: 0.9576     
Epoch 7/100
2289/2289 [==============================] - 0s - loss: 0.1273 - acc: 0.9594     
Epoch 8/100
2289/2289 [==============================] - 0s - loss: 0.1226 - acc: 0.9651     
Epoch 9/100
2289/2289 [==============================] - 0s - loss: 0.1442 - acc: 0.9550     
Epoch 10/100
2289/2289 [==============================] - 0s - loss: 0.1457 - acc: 0.9533     
Epoch 11/100
2289/2289 [==============================] - 0

In [21]:
model_c.save("char.h5")

In [23]:
# test
real = load_labe_c(r"..\Database\charslabel.txt", False)
correct = 0
start = 1110
n = 200
for i in range(start,start+n):
    file = r"..\Database\chars\%d.png" %i
    arr_ = img2arr(file).reshape(1,864)
    p = hotone(model_c.predict(arr_)[0])
    r = real[i]
    if p == r:
        correct += 1
print("correct rate on train set: %.3f" %(correct/n))

correct rate on train set: 0.995


### 2.长度识别

In [24]:
def img2arr_l(imgpath):
    img = Image.open(imgpath).convert("L")
    return np.array(img.getdata())/255

def loaddata_l(path):
    # 由已标定，处理过的图片生成训练集
    lis = os.listdir(path)
    n = len(lis)
    label = np.zeros([n, 2])
    datas = np.zeros([n,10000])
    
    for i,each in enumerate(lis):
        name = each.split(".")[0]
        leng = len(name)
        arry = img2arr(path+"\\"+each)
        datas[i,:] = arry
        label[i, leng-4] = 1
    return (datas,label)

In [25]:
def argmax_l(_mat):
    ans = np.zeros(_mat.shape[0], dtype="int")
    for i in range(_mat.shape[0]):
        ans[i] = np.where(_mat[i] == _mat[i].max())[0][0]        
    return ans

def hotone_l(_arr):
    return np.where(_arr == _arr.max())[0][0]

In [ ]:
# model = Sequential()

# model.add(Dense(100, input_dim=10000, activation='relu', use_bias=True))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu', use_bias=True))
# model.add(Dropout(0.5))
# model.add(Dense(2, activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [28]:
model_l = load_model("length.h5")

#### 训练数据：
_data[i]: 仅经二值化,取反和降噪处理过的原图(未切割)展平的向量_

_labe[i]: onehot型,[1,0] (length=4)或[0,1] (length=5)_

In [29]:
data_l, labe_l = loaddata_l(r"..\Database\proed")

In [30]:
model.fit(data_l, labe_l, epochs=100, batch_size=50)

Epoch 1/100
504/504 [==============================] - 0s - loss: 0.0020 - acc: 1.0000     
Epoch 2/100
504/504 [==============================] - 0s - loss: 0.0021 - acc: 1.0000     
Epoch 3/100
504/504 [==============================] - 0s - loss: 0.0011 - acc: 1.0000     
Epoch 4/100
504/504 [==============================] - 0s - loss: 0.0043 - acc: 1.0000     
Epoch 5/100
504/504 [==============================] - 0s - loss: 0.0016 - acc: 1.0000     
Epoch 6/100
504/504 [==============================] - 0s - loss: 0.0037 - acc: 0.9980     
Epoch 7/100
504/504 [==============================] - 0s - loss: 0.0028 - acc: 1.0000     
Epoch 8/100
504/504 [==============================] - 0s - loss: 0.0016 - acc: 1.0000     
Epoch 9/100
504/504 [==============================] - 0s - loss: 0.0049 - acc: 0.9960     
Epoch 10/100
504/504 [==============================] - 0s - loss: 0.0024 - acc: 1.0000     
Epoch 11/100
504/504 [==============================] - 0s - loss: 0.0038 - acc

In [31]:
model.save("length.h5")

In [32]:
correct = 0
for i in range(439):
    p = argmax_l(model.predict(data_l[i,:].reshape(1,10000)))[0]
    r = hotone_l(labe_l[i])
    if p == r:
        correct += 1
print("correct rate: %.2f" %(correct/439))

correct rate: 1.00


In [14]:
def test(file):
    tempimg = Image.open(file).convert("L")
    tempdata = np.array(tempimg.getdata()).reshape(1,10000)/255
    return argmax(model.predict(tempdata))[0]+4